## LAB9

Write a local-search algorithm (eg. an EA) able to solve the Problem instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

#### Deadlines:
- Submission: Sunday, December 3 (CET)
- Reviews: Sunday, December 10 (CET)

Notes:

- Reviews will be assigned on Monday, December 4
- You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
from random import choices, choice, random, randint, sample
from dataclasses import dataclass
from copy import copy
from tqdm import tqdm
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import lab9_lib

## Genetic Algorithm

In [2]:
GENOME_SIZE = 1000
POPULATION_SIZE = 600
OFFSPRING_SIZE = 450
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .3
CROSSOVER_RATE = .8
NUM_GENERATIONS = 3000
PROBLEM_SIZE = 10
tournament = True

In [3]:
fitness = lab9_lib.make_problem(PROBLEM_SIZE)

In [4]:

for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

00000101010100111001101100011101110101001001011011: 50.00%
11001111101111101111011101001101101110000010110110: 64.00%
10110100100011111001011000100101011101100100001001: 48.00%
00010000000000101001010100001101111111101101110010: 44.00%
10011000011110011000011110111110111111111000010010: 58.00%
00111011100011000011011010000010101101101010001100: 46.00%
11001010011010010011100000111000101101001010011111: 50.00%
00111000010001111100101101110110010100011001010110: 50.00%
01101110111100111111111111101110010010100000010000: 58.00%
10001100010010001100100111100010100110101100111100: 46.00%
10


In [7]:
@dataclass
class Individual:
    genotype: list[int] 
    fitness: tuple 

def select_parent(population):

    if tournament :
        
        
        pool = [choice(population) for _ in range(TOURNAMENT_SIZE)]
        champion = max(pool, key=lambda i: i.fitness)
            
        
        

    else:
        #roulette wheel
        fitness_scores = [fitness(ind.genotype) for ind in population]
        total_fitness = sum(fitness_scores)
        probabilities = [score / total_fitness for score in fitness_scores]
        selected_index = choices(range(len(population)), weights=probabilities)[0]
        champion = population[selected_index]
    
    return champion

# mutation strategies

#select a locus in the genotype and flip the correpsonding bit
def mutate(ind: Individual) -> Individual:
    
    offspring= copy(ind)
    pos = randint(0, GENOME_SIZE-1)
    
    offspring.genotype[pos] = 1 - offspring.genotype[pos]
    offspring.fitness = None
    return  offspring

#for each locus in the genotype flip the correpsonding bit with probability MUTATION_PROBABILITY
def mutate_all(ind: Individual)-> Individual:
    mutated_individual = copy(ind)
    for i in range(len(mutated_individual.genotype)):
        if random() < MUTATION_PROBABILITY:
            mutated_individual.genotype[i] = 1 - mutated_individual.genotype[i]  # Flip bit
    return mutated_individual

#shift each allele right of one position (starting from the last position) and assign to the first postition 
# 0 or 1 with equal probability
def mutate_shift(ind: Individual)-> Individual:
    mutated_individual = copy(ind)
    i=len(mutated_individual.genotype)-1
    while i >= 0:
        if i == 0:
            if random() < 0.5:
                mutated_individual.genotype[i] = 1
            else:
                mutated_individual.genotype[i] = 0
            break
        mutated_individual.genotype[i] = mutated_individual.genotype[i-1]  # shift right by one bit
        
        i-=1

    return mutated_individual



### Crossover startegies

In [8]:
def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, GENOME_SIZE-1)
    offspring1 = Individual(fitness=None,genotype=ind1.genotype[:cut_point]+ind2.genotype[cut_point:])
    offspring2 = Individual(fitness=None,genotype=ind2.genotype[:cut_point]+ind1.genotype[cut_point:])
    
    return offspring1 , offspring2

def  two_cut_xover(ind1: Individual, ind2: Individual)-> Individual:
    point1 = randint(0, GENOME_SIZE-1)
    point2 = randint(0, GENOME_SIZE-1)
    if point2 > point1:
        offspring1 = Individual(fitness=None,genotype=ind1.genotype[:point1] + ind2.genotype[point1:point2] + ind1.genotype[point2:])
        offspring2 = Individual(fitness=None,genotype=ind2.genotype[:point1] + ind1.genotype[point1:point2] + ind2.genotype[point2:])
    else:
        offspring1 = Individual(fitness=None,genotype=ind1.genotype[:point2] + ind2.genotype[point2:point1] + ind1.genotype[point1:])
        offspring2 = Individual(fitness=None,genotype=ind2.genotype[:point2] + ind1.genotype[point2:point1] + ind2.genotype[point1:])
    return offspring1, offspring2


def uniform_crossover(ind1: Individual, ind2: Individual)-> Individual:
    child1 = Individual(fitness=None,genotype=[])
    child2 = Individual(fitness=None,genotype=[])
    
    for gene1, gene2 in zip(ind1.genotype, ind2.genotype):
        if random() < 0.5:
            child1.genotype.append(gene1)
            child2.genotype.append(gene2)
        else:
            child1.genotype.append(gene2)
            child2.genotype.append(gene1)
    return child1, child2

def n_cut_crossover(parent1: Individual, parent2: Individual)-> Individual:
    # Sorting the cut points in ascending order
    n=(GENOME_SIZE//PROBLEM_SIZE) -1
    cut_points = sorted(sample(range(1, len(parent1.genotype)), n))

    child1 = Individual(fitness=None,genotype=[])
    child2 = Individual(fitness=None,genotype=[])

    parents = [parent1,parent2]
    childs = [child1,child2]
    current_parent= 0
    for i in range(len(childs)):
        childs[i].genotype.extend(parents[current_parent].genotype[:cut_points[0]])
        current_parent= 1- current_parent
        for j in range(len(cut_points)):
            if j == (len(cut_points)-1):
                childs[i].genotype.extend(parents[current_parent].genotype[cut_points[j]:])
                if len(cut_points) % 2 == 0:
                    current_parent= 1- current_parent
                    continue
                continue
            childs[i].genotype.extend(parents[current_parent].genotype[cut_points[j]:cut_points[j+1]])
            current_parent= 1- current_parent

    return child1, child2


### Population

In [10]:
def initialize_population():
    

    population = [Individual(genotype=[choice((0,1)) for _ in range(GENOME_SIZE)],fitness=None) for _ in range(POPULATION_SIZE)]

    for i in population:

        i.fitness=fitness(i.genotype)

    print(max(population, key=lambda x : x.fitness).fitness)

    return population

In [11]:
best = [[],0.0]
num_iterations=0

In [12]:
def ga(generational, xover_and_mutation, early_stopping):

    global POPULATION_SIZE
    global OFFSPRING_SIZE 
    global MUTATION_PROBABILITY 
    global CROSSOVER_RATE 
    global NUM_GENERATIONS 
    global PROBLEM_SIZE 
    global TOURNAMENT_SIZE
    global niching
    
    best_fitnesses=[]
    num_iterations=0
    population= initialize_population()
    last_fit = 0.0
    last_5h = 0.0
    for generation in tqdm(range(NUM_GENERATIONS)):
        offspring=list()
        num_iterations+=1
        for counter in range(OFFSPRING_SIZE//2):

            new_off=list()
            if xover_and_mutation:
                #Xover AND mutation
                p1=select_parent(population)
                p2=select_parent(population)
                parents=[p1,p2]
                if random() < CROSSOVER_RATE:     #self adapt mutation probability
                        #add more xovers 
                    o1,o2 = two_cut_xover(p1,p2) 
                    new_off.append(o1)
                    new_off.append(o2)
                    

                if len(new_off):
                    for i in range(len(new_off)):
                        if random() < MUTATION_PROBABILITY:
                            new_off[i]=mutate_shift(new_off[i])
                
                else :
                    for i in range(len(parents)):
                        new_off.append(mutate_shift(parents[i]))

                for os in new_off:   
                    offspring.append(os)
            else:
                #xover OR mutation
                p1=select_parent(population)
                p2=select_parent(population)
                if random() < CROSSOVER_RATE:     
                    
                    o1,o2 = two_cut_xover(p1,p2)                                                     
                    
                else:
                    
                    o1 = mutate(p1)
                    o2 = mutate(p2)
                    
                    
                offspring.append(o1)
                offspring.append(o2)
        
        for i in offspring:
            i.fitness = fitness(i.genotype)
        

        if not generational :
            population.extend(offspring)
            population.sort(key= lambda i: i.fitness , reverse = True)
            population = population[:POPULATION_SIZE]
        
        else:
            #if generational the offspring completely replace the individuals of the past generation
            offspring.sort(key= lambda i: i.fitness , reverse = True)
            population = offspring[:POPULATION_SIZE]
        
        #keep track of the best fitness at every iteration to plot in the end
        best_fitnesses.append(population[0].fitness)

        #update the best absolute fitness found (best[0] is the genotype of the solution, 
        # best[1] is its fitness value)
        if best :
            if population[0].fitness > best[1]:
                best[0] = population[0].genotype
                best[1] = population[0].fitness
        else:
            best[0] = population[0].genotype
            best[1] = population[0].fitness


        #adaptive mutation rate: every 100 generations if there has been little to no upgrade in the 
        # best fitness value, increase the mutation probability
        if (generation +1) % 100 == 0:
            fit_diff = population[0].fitness - last_fit
            last_fit = population[0].fitness
            if fit_diff < 0.01:
                MUTATION_PROBABILITY*=1.1
        
        #stop the algorithm if the best value of the fitness is reached (1) or if there has not been 
        # any (relevant) upgrade in the fitness value last 500 generations
        if early_stopping:
            if best[1]== 1.0 :
                break
            if (num_iterations +1) % 500 == 0:
               fit_diff_5h = population[0].fitness - last_5h
               last_5h = population[0].fitness
               if fit_diff_5h < 0.01:
                   break
                    
    #plot the best fitness value evolution through the generations
    plt.plot(range(1, num_iterations + 1), best_fitnesses)
    plt.title('Problem size : {}, Fitness calls: {}'.format(PROBLEM_SIZE,fitness.calls))
    plt.xlabel('Generation')
    plt.ylabel('Best Fitness')
    plt.show()
    print(fitness.calls)

In [ ]:

ga(generational=False, xover_and_mutation=True, early_stopping=True)

In [ ]:
print(best[1],best[0])

0.406899 [1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 